<a href="https://colab.research.google.com/github/mehtamohit013/comms_lidar_ML/blob/main/BS_Selection/BS_sel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Colab - Only Once

In [ ]:
!nvidia-smi

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
%%capture _
!unzip /gdrive/MyDrive/BTP/Webots_data/train_data.zip
!unzip /gdrive/MyDrive/BTP/Webots_data/val_data.zip
!pip install pytorch-lightning

# Imports

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from enum import Enum

import numpy as np
import os
import matplotlib.pyplot as plt
import random
import scipy.io as io
import math
import getpass

import pytorch_lightning as pl

seed = 0

random.seed(seed)
torch.manual_seed(seed)
np.random.seed(seed)

# Hyper-params

In [ ]:
user = getpass.getuser()

if user == 'root':  #For google colab
    data_dir = '.'
elif user == 'mohit':
    data_dir = os.environ['HOME'] + '/webots_code/data'
elif user == 'iiti':
    data_dir = os.environ['HOME'] + '/webots_code/data'
else:
    print(f'User {user} not present.\n Exiting.....')
    exit(0)

train_dir = os.path.join(data_dir,'train_data')
val_dir = os.path.join(data_dir,'val_data')

#Train Path
train_lpath = os.path.join(train_dir, 'lidar_samples')
train_cpath = os.path.join(train_dir, 'samples')
train_label_path = os.path.join(train_dir,'sample_label')

# Val Path
val_lpath = os.path.join(val_dir, 'lidar_samples')
val_cpath = os.path.join(val_dir, 'samples')
val_label_path = os.path.join(val_dir,'sample_label')


len_train = len(os.listdir(train_lpath))
len_val = len(os.listdir(val_lpath))
BS = np.array([
    [38.89328,-77.07611,5],
    [38.89380,-77.07590,5],
    [38.89393,-77.07644,5]
])
num_BS = int(BS.shape[0])

BATCH_SIZE = 16 

if (user=='root'):
    n_worker = 2
else:
    n_worker = 8

save_dir = os.path.join(os.environ['HOME'],'webots_code/model_state_dict')
os.makedirs(save_dir,exist_ok=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


# Utilities

In [ ]:
def top_k_acc(y_true:torch.Tensor,y_pred:torch.Tensor,k=1):
    
    y_pred_tpk = torch.topk(y_pred,k,dim=1)[1]
    
    ovr = 0
    pos = 0

    for i in range(0,len(y_pred_tpk)):
        if(y_true[i] in y_pred_tpk[i]):
            pos+=1
        ovr+=1
    
    acc = pos/ovr
    return acc

# Creating Dataset and Dataloader

## Custom Dataset
Using 'GPS' currently.Translation can also be used instead of gps

In [ ]:
class bs_dataset(Dataset):
    def __init__(self,lpath:str,cpath:str,
                 label_path:str,len_data:int,BS:np.ndarray):
        
        self.lpath = lpath
        self.cpath = cpath
        self.label_path = label_path
        self.len_data = len_data
        self.BS = BS #[num_BS,3]
        
    def __getitem__(self,idx):
        sample = dict()
        
        sample['lidar'] = dict(np.load(self.lpath+f'/{idx}.npz'))['lidar'] #[10,240,240]
        sample['lidar'] = sample['lidar'].astype('float32')
        sample['gps'] = io.loadmat(self.cpath+f'/{idx}.mat')['gps']
        sample['gps'] = sample['gps'].astype('float32').reshape((3,)) #[3,]
        sample['BS'] = self.BS.reshape((3*num_BS,)).astype('float32') #[num_BS*3,]
        sample['label'] = io.loadmat(self.label_path+f'/{idx}.mat')['ss']
        
        #Return the index of maximum element 
        sample['label'] = np.argmax(sample['label'].astype('float32')) 
        
        return sample
    
    def __len__(self):
        return self.len_data

In [ ]:
train_dataset = bs_dataset(train_lpath,train_cpath,train_label_path,len_train,BS)

In [ ]:
val_dataset = bs_dataset(val_lpath,val_cpath,val_label_path,len_val,BS)

## Dataloader

In [ ]:
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    pin_memory=True,
    num_workers = n_worker,
    drop_last = True,
    shuffle = True
    )

In [ ]:
val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    pin_memory=True,
    num_workers = n_worker,
    drop_last = True
    )

# Models
Based on Imperial Model

## Lidar + GPS Class

In [ ]:
class bs_model(nn.Module):
    def __init__(self):
        super().__init__()
        self.channels = 5
        self.fchannel = 3
        self.conv1 = nn.Conv2d(10,self.channels, 13, 1, 1)
        self.bn1 = nn.BatchNorm2d(self.channels)
        self.relu1 = nn.PReLU(num_parameters=self.channels)
        self.conv2 = nn.Conv2d(self.channels, self.channels, 13, 1, 1)
        self.bn2 = nn.BatchNorm2d(self.channels)
        self.relu2 = nn.PReLU(num_parameters=self.channels)
        self.conv3 = nn.Conv2d(self.channels, self.channels, 7, 2, 1)
        self.bn3 = nn.BatchNorm2d(self.channels)
        self.relu3 = nn.PReLU(num_parameters=self.channels)
        self.conv4 = nn.Conv2d(self.channels, self.channels, 7, 1, 1)
        self.bn4 = nn.BatchNorm2d(self.channels)
        self.relu4 = nn.PReLU(num_parameters=self.channels)
        self.conv5 = nn.Conv2d(self.channels, self.fchannel, 5, 2, 1)
        self.bn5 = nn.BatchNorm2d(self.fchannel)
        self.relu5 = nn.PReLU(num_parameters=self.fchannel)
        self.conv6 = nn.Conv2d(self.fchannel, self.fchannel, 5, (1, 2), 1)
        self.bn6 = nn.BatchNorm2d(self.fchannel)
        self.relu6 = nn.PReLU(num_parameters=self.fchannel)
        
        self.flatten = nn.Flatten()
        self.linear7 = nn.Linear(3675,64)
        self.relu7 = nn.ReLU()
        self.linear8 = nn.Linear(76, 3)
#         self.linear9 = nn.Linear(64,3)
#         self.linear10 = nn.Linear(16,3)


    def forward(self, x, gps, bs):

        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)

        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu2(x)

        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu3(x)
        #
        x = self.conv4(x)
        x = self.bn4(x)
        x = self.relu4(x)

        x = self.conv5(x)
        x = self.bn5(x)
        x = self.relu5(x)

        x = self.conv6(x)
        x = self.bn6(x)
        x = self.relu6(x)

        x = self.flatten(x)
        x = self.linear7(x)
        x = self.relu7(x)
        
        out = torch.cat((x,gps,bs),dim=1)
        out = self.linear8(out)
#         out = self.relu7(out)
#         out = self.linear9(out)
#         out = self.relu7(out)
#         out = self.linear10(out)
        
        return out


In [ ]:
# # For testing model shape and size
# model = bs_model()lidar = torch.Tensor(2,10,240,240)
# gps = torch.Tensor(2,3)
# BS = torch.Tensor(2,9)
# model(lidar,gps,BS)

## GPS Class

In [ ]:
class gps_bs_sel(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.linear1 = nn.Linear(12,32)
        self.linear2 = nn.Linear(32,64)
        self.linear3 = nn.Linear(64,32)
        self.linear4 = nn.Linear(32,3)
        
        self.relu = nn.ReLU()
    
    def forward(self,gps,BS):

        x = torch.cat([gps,BS],dim=1)
        
        out = self.linear1(x)
        out = self.relu(out)
        out = self.linear2(out)
        out = self.relu(out)
        out = self.linear3(out)
        out = self.relu(out)
        out = self.linear4(out)

        return out

# Training
Using pytorch lightning

## GPS

### Lightning class

In [ ]:
class gps_trainer(pl.LightningModule):
    def __init__(self,learning_rate = 1e-3):
        super().__init__()
        self.model = gps_bs_sel()
        self.celoss = nn.CrossEntropyLoss()
        self.lr = learning_rate
    
    def forward(self,gps,BS):
        out = self.model(gps,BS)
        return out
    
    def training_step(self,batch,batch_idx):
        gps = batch['gps'].float()
        BS = batch['BS'].float()
        label = batch['label'].long()
        
        yhat = self(gps,BS)
        
        loss = self.celoss(yhat,label)
        
        self.log('my_loss',loss)
        
        return {'loss':loss,'pred':yhat.cpu().detach(),'label':label.cpu().detach()}
    
    def configure_optimizers(self):
        opt = torch.optim.Adam(self.parameters(),self.lr)
        return opt
    
    def training_epoch_end(self,train_out):        
        len_out = len(train_out)
        y_pred = torch.Tensor(len_out*BATCH_SIZE,num_BS)
        y_true = torch.Tensor(len_out*BATCH_SIZE)

        for i in range(0,len_out):
            y_pred[i*BATCH_SIZE:(i+1)*BATCH_SIZE,:] = train_out[i]['pred'] 
            y_true[i*BATCH_SIZE:(i+1)*BATCH_SIZE] = train_out[i]['label']

        top1 = top_k_acc(y_true,y_pred,k=1)

        print(f'Train accuracies is {top1}')

    def validation_step(self,batch,batch_idx):
        gps = batch['gps'].float()
        BS = batch['BS'].float()
        label = batch['label'].long()
        
        yhat = self.forward(gps,BS)
        
        return [yhat.cpu().detach(),label.cpu().detach()]
     
    def validation_epoch_end(self,val_out):
        len_out = len(val_out)
        y_pred = torch.Tensor(len_out*BATCH_SIZE,num_BS)
        y_true = torch.Tensor(len_out*BATCH_SIZE)

        for i in range(0,len_out):
            y_pred[i*BATCH_SIZE:(i+1)*BATCH_SIZE,:] = val_out[i][0] 
            y_true[i*BATCH_SIZE:(i+1)*BATCH_SIZE] = val_out[i][1] 

        top1 = top_k_acc(y_true,y_pred,k=1)
        print(f'Validation accuracy is {top1}')
    
    def test_step(self,batch,batch_idx):
        gps = batch['gps'].float()
        BS = batch['BS'].float()
        label = batch['label'].long()
        
        yhat = self.forward(gps,BS)
        
        return [yhat.cpu().detach(),label.cpu().detach()]
     
    def test_epoch_end(self,val_out):
        len_out = len(val_out)
        y_pred = torch.Tensor(len_out*BATCH_SIZE,num_BS)
        y_true = torch.Tensor(len_out*BATCH_SIZE)

        for i in range(0,len_out):
            y_pred[i*BATCH_SIZE:(i+1)*BATCH_SIZE,:] = val_out[i][0] 
            y_true[i*BATCH_SIZE:(i+1)*BATCH_SIZE] = val_out[i][1] 

        top1 = top_k_acc(y_true,y_pred,k=1)
        print(f'Test accuracy is {top1}')

### Class object and trainer

In [ ]:
gps_model = gps_trainer()

In [ ]:
gps_pl_trainer = pl.Trainer(reload_dataloaders_every_epoch = True,
                     gpus=1,
                     max_epochs = 10,
                     auto_lr_find = False
                     )

In [ ]:
gps_pl_trainer.fit(gps_model,train_loader,val_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type             | Params
--------------------------------------------
0 | model  | gps_bs_sel       | 4.7 K 
1 | celoss | CrossEntropyLoss | 0     
--------------------------------------------
4.7 K     Trainable params
0         Non-trainable params
4.7 K     Total params
0.019     Total estimated model params size (MB)


Validation accuracy is 0.25


Train accuracies is 0.5728571428571428


Validation accuracy is 0.6354166666666666
Train accuracies is 0.5758928571428571


Validation accuracy is 0.6354166666666666
Train accuracies is 0.58125


...

Validation accuracy is 0.6354166666666666
Train accuracies is 0.58125


Validation accuracy is 0.6354166666666666
Train accuracies is 0.58125


Validation accuracy is 0.6354166666666666
Train accuracies is 0.5810714285714286


Validation accuracy is 0.6354166666666666
Train accuracies is 0.58125


Validation accuracy is 0.6354166666666666



##  GPS + Lidar

### Lightning Class

In [ ]:
class BS_trainer(pl.LightningModule):
    def __init__(self,learning_rate = 1e-3):
        super().__init__()
        self.model = bs_model()
        self.celoss = nn.CrossEntropyLoss()
        self.lr = learning_rate
    
    def forward(self,lidar,gps,BS):
        out = self.model(lidar,gps,BS)
        return out
    
    def training_step(self,batch,batch_idx):
        lidar = batch['lidar'].float()
        gps = batch['gps'].float()
        BS = batch['BS'].float()
        label = batch['label'].long()
        
        yhat = self(lidar,gps,BS)
        
        loss = self.celoss(yhat,label)
        
        self.log('my_loss',loss)
        
        return {'loss':loss,'pred':yhat.cpu().detach(),'label':label.cpu().detach()}
    
    def configure_optimizers(self):
        opt = torch.optim.Adam(self.parameters(),self.lr)
        return opt
    
    def training_epoch_end(self,train_out):        
        len_out = len(train_out)
        y_pred = torch.Tensor(len_out*BATCH_SIZE,num_BS)
        y_true = torch.Tensor(len_out*BATCH_SIZE)

        for i in range(0,len_out):
            y_pred[i*BATCH_SIZE:(i+1)*BATCH_SIZE,:] = train_out[i]['pred'] 
            y_true[i*BATCH_SIZE:(i+1)*BATCH_SIZE] = train_out[i]['label']

        top1 = top_k_acc(y_true,y_pred,k=1)

        print(f'Train accuracies is {top1}')

    def validation_step(self,batch,batch_idx):
        lidar = batch['lidar'].float()
        gps = batch['gps'].float()
        BS = batch['BS'].float()
        label = batch['label'].long()
        
        yhat = self.forward(lidar,gps,BS)
        
        return [yhat.cpu().detach(),label.cpu().detach()]
     
    def validation_epoch_end(self,val_out):
        len_out = len(val_out)
        y_pred = torch.Tensor(len_out*BATCH_SIZE,num_BS)
        y_true = torch.Tensor(len_out*BATCH_SIZE)

        for i in range(0,len_out):
            y_pred[i*BATCH_SIZE:(i+1)*BATCH_SIZE,:] = val_out[i][0] 
            y_true[i*BATCH_SIZE:(i+1)*BATCH_SIZE] = val_out[i][1] 

        top1 = top_k_acc(y_true,y_pred,k=1)
        print(f'Validation accuracy is {top1}')
    
    def test_step(self,batch,batch_idx):
        lidar = batch['lidar'].float()
        gps = batch['gps'].float()
        BS = batch['BS'].float()
        label = batch['label'].long()
        
        yhat = self.forward(lidar,gps,BS)
        
        return [yhat.cpu().detach(),label.cpu().detach()]
     
    def test_epoch_end(self,val_out):
        len_out = len(val_out)
        y_pred = torch.Tensor(len_out*BATCH_SIZE,num_BS)
        y_true = torch.Tensor(len_out*BATCH_SIZE)

        for i in range(0,len_out):
            y_pred[i*BATCH_SIZE:(i+1)*BATCH_SIZE,:] = val_out[i][0] 
            y_true[i*BATCH_SIZE:(i+1)*BATCH_SIZE] = val_out[i][1] 

        top1 = top_k_acc(y_true,y_pred,k=1)
        print(f'Test accuracy is {top1}')

### Class object and trainer 

In [ ]:
model = BS_trainer()

In [ ]:
lidar_pl_trainer = pl.Trainer(reload_dataloaders_every_epoch = True,
                     gpus=1,
                     max_epochs = 10,
                     auto_lr_find = False
                     )

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [ ]:
lidar_pl_trainer.fit(model,train_loader,val_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   | Type             | Params
--------------------------------------------
0 | model  | bs_model         | 251 K 
1 | celoss | CrossEntropyLoss | 0     
--------------------------------------------
251 K     Trainable params
0         Non-trainable params
251 K     Total params
1.005     Total estimated model params size (MB)


Validation accuracy is 0.0


Train accuracies is 0.8091071428571428


Validation accuracy is 0.9375
Train accuracies is 0.8555357142857143


Validation accuracy is 0.9375
Train accuracies is 0.8758928571428571


...

Validation accuracy is 0.9270833333333334
Train accuracies is 0.9219642857142857


Validation accuracy is 0.9166666666666666
Train accuracies is 0.9310714285714285


Validation accuracy is 0.9166666666666666
Train accuracies is 0.9391071428571428


Validation accuracy is 0.90625
Train accuracies is 0.9558928571428571


Validation accuracy is 0.90625



# Accuracy based on distance

In [ ]:
def dist_gps(gps1, gps2):
    lat1, lon1, _ = gps1
    lat2, lon2, _ = gps2
    R = 6371000  # radius of Earth in meters
    phi_1 = math.radians(lat1)
    phi_2 = math.radians(lat2)

    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)

    a = math.sin(delta_phi / 2.0) ** 2 + \
        math.cos(phi_1) * math.cos(phi_2) * \
        math.sin(delta_lambda / 2.0) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c


In [ ]:
y_pred = list()
pos = 0
ovr = 0 
for i in range(0,len_train):
    data = train_dataset[i]
    dist1 = dist_gps(data['gps'],data['BS'][:3])
    dist2 = dist_gps(data['gps'],data['BS'][3:6])
    dist3 = dist_gps(data['gps'],data['BS'][6:9])

    # print(dist1,dist2,dist3)
    
    index = np.argmax(np.array([dist1,dist2,dist3]))

    # print(index,data['label'])

    if index == data['label']:
        pos+=1
    ovr+=1
    # if((i+1)%10==0):
    #     break
print(f'Accuracy based on shortest distance on train is {pos/ovr}')

Accuracy based on shortest distance on train is 0.2945902517407606


In [ ]:
y_pred = list()
pos = 0
ovr = 0 
for i in range(0,len_val):
    data = val_dataset[i]
    dist1 = dist_gps(data['gps'],data['BS'][:3])
    dist2 = dist_gps(data['gps'],data['BS'][3:6])
    dist3 = dist_gps(data['gps'],data['BS'][6:9])

    # print(dist1,dist2,dist3)
    
    index = np.argmax(np.array([dist1,dist2,dist3]))

    # print(index,data['label'])

    if index == data['label']:
        pos+=1
    ovr+=1
    # if((i+1)%10==0):
    #     break
print(f'Accuracy based on shortest distance on val is {pos/ovr}')

Accuracy based on shortest distance on val is 0.29357798165137616
